# Conditional MC of 3/2 Model based on QE Scheme

In [1]:
import numpy as np
import sv32_qe_cmc as sv32
import time
import pyfeng as pf
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# parameters
strike = [100.0, 140.0, 70.0]
forward = 100
delta = [1, 1/2, 1/4, 1/8, 1/16, 1/32]

case = np.zeros([3, 6])
case[0] = [1,   0.5, -0.9, 10, 0.04, np.sqrt(0.04)]
case[1] = [0.9, 0.3, -0.5, 15, 0.04, np.sqrt(0.04)]
case[2] = [1,   1,   -0.3, 5,  0.09, np.sqrt(0.09)]

In [3]:
# compute cmc price
for i in range(3):
    vov, kappa, rho, texp, theta, sigma = case[i]
    start = time.time()

    sv32_cmc_qe = sv32.SV32QECondMC(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike)])
    for d in range(len(delta)):
        price_cmc[d, :] = sv32_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e5, seed=123456)

    end = time.time()
    
    print('Case %s:' % i)
    print(price_cmc)
    print('Running time is %.3f seconds.' % (end - start) + '\n')

Case 0:
[[22.95314785 10.44093783 38.98364955]
 [23.2425543  10.67162543 39.26731165]
 [23.20965635 10.64143576 39.21865023]
 [22.93527518 10.4758762  38.87971674]
 [22.9298084  10.47613694 38.88556212]
 [23.12806844 10.56484306 39.16893668]]
Running time is 27.734 seconds.

Case 1:
[[28.38562985 16.58649523 42.55740431]
 [28.42977766 16.61705767 42.61118032]
 [28.30786768 16.53990973 42.44717881]
 [28.30763704 16.53355657 42.45931412]
 [28.2822572  16.51219277 42.43468726]
 [28.37245592 16.57440991 42.54708381]]
Running time is 36.096 seconds.

Case 2:
[[25.20424465 13.66791193 39.93642084]
 [25.17249128 13.64813567 39.8960167 ]
 [25.20448092 13.66501972 39.94017581]
 [25.19447764 13.66509173 39.91560506]
 [25.12647967 13.62566786 39.82080577]
 [25.14555403 13.63602067 39.85066992]]
Running time is 11.482 seconds.



In [4]:
# compute std error of cmc price
n = 50
for i in range(3):
    start = time.time()
    vov, kappa, rho, texp, theta, sigma = case[i]

    sv32_cmc_qe = sv32.SV32QECondMC(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike), n])
    for j in tqdm(range(n)):
        for d in range(len(delta)):
            price_cmc[d, :, j] = sv32_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e4)

    end = time.time()
    np.set_printoptions(suppress=True)
    print('Case %s:' % i)
    print(price_cmc.mean(axis=2))
    print(price_cmc.std(axis=2))
    print('Running time is %.3f seconds.' % (end - start) + '\n')

 70%|█████████████████████████████████████████████████████████▍                        | 35/50 [01:18<00:32,  2.16s/it]C:\Python\Anaconda3\lib\site-packages\pyfeng\bsm.py:54: RuntimeWarning: divide by zero encountered in log
  d1 = np.log(fwd / strike) / sigma_std
100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:50<00:00,  2.22s/it]


Case 0:
[[23.08755832 10.54413041 39.12273369]
 [23.06171059 10.5377992  39.06402042]
 [23.09132568 10.5526548  39.10527713]
 [23.1063742  10.57337139 39.11076507]
 [23.060259   10.55044323 39.05412788]
 [23.09456981 10.53632628 39.13327679]]
[[0.38627878 0.25925548 0.47355052]
 [0.36605422 0.23499384 0.48579429]
 [0.36137381 0.22991383 0.46211131]
 [0.29660141 0.19994536 0.37871732]
 [0.33553104 0.22650694 0.39766623]
 [0.31130555 0.21975281 0.37122441]]
Running time is 110.781 seconds.



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:42<00:00,  3.26s/it]


Case 1:
[[28.38176415 16.58057301 42.55525935]
 [28.35166261 16.55903028 42.52130657]
 [28.34925288 16.56135353 42.51136443]
 [28.33490703 16.5485706  42.49867665]
 [28.33550154 16.54656972 42.50290488]
 [28.35327862 16.56283688 42.51750731]]
[[0.21698265 0.14289423 0.28302751]
 [0.15812468 0.10728795 0.20349458]
 [0.16607334 0.1066586  0.22379982]
 [0.17963158 0.11966924 0.23404913]
 [0.19341311 0.12517267 0.2585399 ]
 [0.1790967  0.12076539 0.23121264]]
Running time is 162.879 seconds.



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:57<00:00,  1.14s/it]


Case 2:
[[25.16429458 13.64516993 39.87973014]
 [25.19559174 13.66095937 39.92777535]
 [25.18403963 13.65618285 39.90780783]
 [25.1902598  13.66131962 39.91425366]
 [25.16157042 13.64605559 39.87089809]
 [25.18803747 13.65919727 39.91321666]]
[[0.08024194 0.04334451 0.11968199]
 [0.0939781  0.05331117 0.13512865]
 [0.0976523  0.0561717  0.13951291]
 [0.10046091 0.05507797 0.14949366]
 [0.08965092 0.05055121 0.13105388]
 [0.08540538 0.0454929  0.12674164]]
Running time is 57.146 seconds.

